In [1]:
from topagnps2cche1d import Node, Reach, Watershed
from topagnps2cche1d.tools import get_intermediate_nodes_img

In [2]:
n1 = Node(id=1, x=0, y=0)
n2 = Node(id=2, x=1, y=1)

n1.distance_from(n2)

1.4142135623730951

In [3]:
import networkx as nx

graph = nx.DiGraph()

edges = [(4, 5), (3, 5), (5, 9), (8, 9), (6, 8), (7, 8), (9, 1), (2, 1)]

graph.add_edges_from(edges)

In [4]:
nx.ancestors(graph, 5)

{3, 4}